In [ ]:
!pip install jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import bokeh.plotting as bp
from bokeh.models import HoverTool
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.layouts import row

pn.extension()

# Load temperature data
url_rcp45 = "https://raw.githubusercontent.com/rodrigourech/ivi/main/data/RCP45_tas.csv"
url_rcp85 = "https://raw.githubusercontent.com/rodrigourech/ivi/main/data/RCP85_tas.csv"

df_rcp45 = pd.read_csv(url_rcp45, parse_dates=["date"])
df_rcp85 = pd.read_csv(url_rcp85, parse_dates=["date"])

df_rcp45["scenario"] = "RCP45"
df_rcp85["scenario"] = "RCP85"
df_all = pd.concat([df_rcp45, df_rcp85], ignore_index=True)
df_all["year"] = df_all["date"].dt.year


In [ ]:
pn.extension()

# Widgets
station_select = pn.widgets.Select(
    name='Station',
    options=sorted(df_all["station"].unique()),
    width=200
)

year_slider = pn.widgets.IntRangeSlider(
    name='Year Range',
    start=1981,
    end=2099,
    value=(1981, 2100),
    step=1,
    width=800,
    styles={'font-size': '12pt'}  # Verbessert Lesbarkeit der aktiven Range
)

smooth_checkbox = pn.widgets.Checkbox(
    name='Apply 30-day Rolling Average',
    value=False,
    width=200
)

dual_line_checkbox = pn.widgets.Checkbox(
    name='Compare RCP45 vs. RCP85',
    value=False,
    width=200
)

# Plot function
@pn.depends(station=station_select, year_range=year_slider,
            smooth=smooth_checkbox, dual=dual_line_checkbox)
def update_plots(station, year_range, smooth, dual):
    df_filtered = df_all[
        (df_all["station"] == station) &
        (df_all["year"] >= year_range[0]) &
        (df_all["year"] <= year_range[1])
    ]

    p = figure(
        x_axis_type="datetime",
        title=f"Daily Temperatures in {station} ({year_range[0]}–{year_range[1]})",
        width=1200,
        height=400,
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )
    p.yaxis.axis_label = "Temperature (°C)"
    p.xaxis.axis_label = "Date"

    hist = figure(
        title="Temperature Distribution",
        width=700,
        height=400,
        tools="reset,save"
    )
    hist.xaxis.axis_label = "Temperature (°C)"
    hist.yaxis.axis_label = "Frequency"

    scenarios = ["RCP45", "RCP85"] if dual else [df_filtered["scenario"].iloc[0]]
    colors = {"RCP45": "#0072B2", "RCP85": "#D55E00"}

    for scenario in scenarios:
        data_orig = df_filtered[df_filtered["scenario"] == scenario].sort_values("date")
        data_line = data_orig.copy()
        if smooth:
            data_line["value"] = data_line["value"].rolling(window=30, min_periods=1).mean()
        p.line(
            data_line["date"],
            data_line["value"],
            line_width=2,
            color=colors[scenario],
            legend_label=scenario
        )

        # Histogram
        hist_data = data_orig["value"].dropna()
        hist_values, edges = np.histogram(hist_data, bins=50)
        hist.quad(top=hist_values, bottom=0, left=edges[:-1], right=edges[1:],
                  fill_color=colors[scenario], line_color="white", alpha=0.6, legend_label=scenario)

    p.add_tools(HoverTool(
        tooltips=[("Date", "@x{%F}"), ("Temp", "@y{0.1f} °C")],
        formatters={"@x": "datetime"},
        mode='vline'
    ))
    p.legend.location = "top_left"
    hist.legend.location = "top_right"

    return row(p, hist)

# Dashboard layout (angepasst für bessere Usability)
dashboard = pn.Column(
    pn.pane.Markdown("## Climate Scenario Explorer – LO2 Dashboard"),
    pn.Row(station_select, year_slider),
    pn.Row(pn.Column(smooth_checkbox, dual_line_checkbox)),
    update_plots
)

dashboard.servable()


Column
    [0] Markdown(str)
    [1] Row
        [0] Select(name='Station', options=['Basel', 'Bern', ...], value='Basel', width=200)
        [1] IntRangeSlider(end=2099, name='Year Range', start=1981, styles={'font-size': '12pt'}, value=(1981, 2100), value_end=2100, value_start=1981, width=800)
    [2] Row
        [0] Column
            [0] Checkbox(name='Apply 30-day R..., width=200)
            [1] Checkbox(name='Compare RCP45 vs. RCP85', width=200)
    [3] ParamFunction(function, _pane=Bokeh, defer_load=False)

In [2]:
import pandas as pd
import numpy as np
import panel as pn
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, LabelSet

pn.extension()

# Widgets
station_select = pn.widgets.Select(
    name='',
    options=sorted(df_all["station"].unique()),
    width=200
)

scenario_select = pn.widgets.RadioButtonGroup(
    name='',
    options=['RCP45', 'RCP85'],
    button_type='success',
    width=200
)

dual_line_checkbox = pn.widgets.Checkbox(
    name='Compare both scenarios',
    value=False,
    width=220
)

year_slider = pn.widgets.IntRangeSlider(
    name='Year Range',
    start=1981,
    end=2099,
    value=(1981, 2100),
    step=1,
    width=800,
    styles={'font-size': '12pt'}
)

# Callback
@pn.depends(
    station=station_select,
    scenario=scenario_select,
    year_range=year_slider,
    dual=dual_line_checkbox
)
def update_plots(station, scenario, year_range, dual):
    df_filtered = df_all[
        (df_all["station"] == station) &
        (df_all["year"] >= year_range[0]) &
        (df_all["year"] <= year_range[1])
    ]

    # Plot 1: Daily Temperature
    p = figure(
        x_axis_type="datetime",
        title=f"Daily Temperatures (30-Day Rolling Avg)",
        width=1200,
        height=400,
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )
    p.yaxis.axis_label = "Temperature (°C)"
    p.xaxis.axis_label = "Date"

    # Plot 2: Histogram
    hist = figure(
        title="Temperature Distribution",
        width=700,
        height=400,
        tools="reset,save"
    )
    hist.xaxis.axis_label = "Temperature (°C)"
    hist.yaxis.axis_label = "Frequency"

    # Plot 3: Trend Summary
    trend = figure(
        title="Annual Mean Temperature (5-Year Rolling Avg)",
        width=900,
        height=300,
        x_axis_type="linear",
        tools="pan,box_zoom,reset,save"
    )
    trend.xaxis.axis_label = "Year"
    trend.yaxis.axis_label = "Temperature (°C)"

    # Plot 4: Temperature Increase Summary
    delta_fig = figure(
        x_range=[],
        title=f"Temperature Increase",
        width=500,
        height=300,
        tools="save"
    )
    delta_fig.yaxis.axis_label = "Δ Temperature (°C)"

    scenarios = ["RCP45", "RCP85"] if dual else [scenario]
    colors = {"RCP45": "#0072B2", "RCP85": "#D55E00"}
    delta_data = {"scenario": [], "delta": [], "color": [], "label": [], "y_pos": []}

    for scen in scenarios:
        data_orig = df_filtered[df_filtered["scenario"] == scen].sort_values("date")
        data_line = data_orig.copy()
        # 30-day rolling average applied by default
        data_line["value"] = data_line["value"].rolling(window=30, min_periods=1).mean()

        # Daily Plot
        p.line(
            data_line["date"],
            data_line["value"],
            line_width=2,
            color=colors[scen],
            legend_label=scen
        )

        # Histogram
        hist_data = data_orig["value"].dropna()
        hist_values, edges = np.histogram(hist_data, bins=50)
        hist.quad(top=hist_values, bottom=0, left=edges[:-1], right=edges[1:],
                  fill_color=colors[scen], line_color="white", alpha=0.6, legend_label=scen)

        # Trend Summary – 5-year rolling mean
        df_yearly = data_orig.groupby(data_orig["date"].dt.year)["value"].mean().reset_index()
        df_yearly.columns = ["year", "mean"]
        df_yearly["smoothed"] = df_yearly["mean"].rolling(window=5, min_periods=1).mean()

        source = ColumnDataSource(df_yearly)
        trend.line("year", "smoothed", source=source, line_width=2, color=colors[scen], legend_label=scen)

        # Δ Temperature
        start_temp = df_yearly["smoothed"].iloc[0]
        end_temp = df_yearly["smoothed"].iloc[-1]
        delta_temp = end_temp - start_temp

        delta_data["scenario"].append(scen)
        delta_data["delta"].append(round(delta_temp, 2))
        delta_data["color"].append(colors[scen])
        delta_data["label"].append(f"{delta_temp:+.2f} °C")
        delta_data["y_pos"].append(round(delta_temp / 2, 2))

    # Delta bar chart
    if delta_data["scenario"]:
        delta_source = ColumnDataSource(delta_data)
        delta_fig.x_range.factors = delta_data["scenario"]
        delta_fig.vbar(x="scenario", top="delta", width=0.5, color="color", source=delta_source)

        labels = LabelSet(
            x='scenario', y='y_pos', text='label',
            level='glyph', x_offset=0, y_offset=0,
            text_align='center', text_baseline='middle',
            text_font_size="12pt", text_color="white",
            background_fill_color="black", background_fill_alpha=1.0,
            source=delta_source
        )
        delta_fig.add_layout(labels)

    # Tooltips
    p.add_tools(HoverTool(
        tooltips=[("Date", "@x{%F}"), ("Temp", "@y{0.1f} °C")],
        formatters={"@x": "datetime"},
        mode='vline'
    ))
    trend.add_tools(HoverTool(
        tooltips=[("Year", "@year"), ("Mean (5Y)", "@smoothed{0.2f} °C")],
        mode='vline'
    ))

    p.legend.location = "top_left"
    hist.legend.location = "top_right"
    trend.legend.location = "top_left"

    return pn.Column(
        pn.Row(p, hist),
        pn.Row(trend, delta_fig)
    )

# Layout
dashboard = pn.Column(
    pn.pane.Markdown("## Climate Scenario Explorer (v2)"),
    pn.Row(
        pn.Column(pn.pane.Markdown("### Station"), station_select),
        pn.Column(pn.pane.Markdown("### Scenario"), scenario_select),
        pn.Column(pn.pane.Markdown("### Settings"), dual_line_checkbox)
    ),
    year_slider,
    update_plots
)

dashboard.servable()


/tmp/ipython-input-2-691469389.py:7: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension()


NameError: name 'df_all' is not defined